In [1]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.3Rw6alhmxs/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.EIUaLHYq5D/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.Ms0qgeAUbt/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [2]:
!apt-get update
!apt-get install chromium chromium-driver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:11 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InR

In [3]:
!pip install Selenium==4.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

# from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# from selenium.common.exceptions import ElementClickInterceptedException

from time import sleep

import numpy as np

In [49]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox') # needed, because colab runs as root
    options.add_argument('--headless')  # or use pyvirtualdiplay
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

In [34]:
import glob
import os
from google.colab import files

def descarrega_xlsx(nou_nom):

    # Busca tots els arxius amb extensió .xlsx a la carpeta arrel de Colab
    xlsx_files = glob.glob('*.xlsx')

    if not xlsx_files:
        print("No s'han trobat arxius .xlsx")
        return

    # Ordena els arxius per data de modificació (del més recent al més antic)
    xlsx_files.sort(key=os.path.getmtime, reverse=True)

    # Agafa el primer arxiu (el més recent)
    ultim_arxiu = xlsx_files[0]

    nou_nom_complet = nou_nom + ".xlsx"

    os.rename(ultim_arxiu, nou_nom_complet)


In [51]:
from urllib.parse import quote
import glob
from google.colab import files
import time

def titular(codi):

  driver = web_driver()

  # URL del lloc web
  url = "https://branddb.wipo.int/en/advancedsearch/results?sort=score%20desc&strategy=concept&asStructure=%7B%22_id%22:%22aff0%22,%22boolean%22:%22AND%22,%22bricks%22:%5B%7B%22_id%22:%22aff1%22,%22key%22:%22applicant%22,%22strategy%22:%22Simple%22,%22value%22:%22%5C%22"+quote(codi)+"%5C%22%22%7D%5D%7D&_=1720017059217&fg=_void_"

  # Obre el lloc web
  driver.get(url)

  try:

    # Defineix l'XPath de l'element
    element_xpath = "/html/body/app-root/div/div/page-results/results/div/results-info/div[3]/div"

    # Espera fins que l'element sigui visible
    wait = WebDriverWait(driver, 120)
    element = wait.until(EC.visibility_of_element_located((By.XPATH, element_xpath)))

    # Continua verificant el text fins que deixi de ser "Loading..."
    try:
        wait.until_not(lambda driver: driver.find_element(By.XPATH, element_xpath).text == "Loading...")
        # print("La càrrega s'ha completat.")
    except TimeoutException:
        print(f"L'element encara mostra 'Loading...' després de 120 segons.")


    # Espera que el botó "Download results" sigui visible i clicable
    wait = WebDriverWait(driver, 1)
    download_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//span[text()="Download results"]')))
    download_button.click()

    wait = WebDriverWait(driver, 5)
    # Espera que l'opció "EXCEL" sigui visible i clicable
    excel_option = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/app-root/div/div/page-results/results/div/search-results-menu/div/w-table-commands/ul/f-facet[4]/li/div/div[2]/span')))
    excel_option.click()

    # Dona temps perquè la descàrrega es completi
    time.sleep(25)  # Ajusta aquest temps segons sigui necessari

    descarrega_xlsx(codi)
    print("   S'ha descarregat la informació de l'entitat", codi)
    return codi, "1"

  except:
    print("   No s'ha trobat l'entitat", codi)
    time.sleep(5)
    return codi, "0"


In [25]:
import pandas as pd

df = pd.read_excel("entitats.xlsx")

df2 = df['Titular (entitat)'].apply(lambda x: x.rsplit(",", 1)[0])
df2 = df2.apply(lambda x: x.rsplit(' (', 1)[0])

In [ ]:
log = []
for x in df2[128:]:
  print(x)
  entitat, status = titular(x)
  log.append([entitat, status])

Fundació Biblioteca Josep Pla
   No s'ha trobat l'entitat Fundació Biblioteca Josep Pla
Fundació Bonanova
   No s'ha trobat l'entitat Fundació Bonanova
Fundació Carles Pi i Sunyer
   S'ha descarregat la informació de l'entitat Fundació Carles Pi i Sunyer
Fundació Casa Amèrica Catalunya
   S'ha descarregat la informació de l'entitat Fundació Casa Amèrica Catalunya
Fundació Casa Cresques de Barcelona
   No s'ha trobat l'entitat Fundació Casa Cresques de Barcelona
Fundació Catalana per a la Recerca i la Innovació
   S'ha descarregat la informació de l'entitat Fundació Catalana per a la Recerca i la Innovació
Fundació Catalana per a l'Ensenyament de l'Idioma Anglès i l'Educació en Anglès
   No s'ha trobat l'entitat Fundació Catalana per a l'Ensenyament de l'Idioma Anglès i l'Educació en Anglès
Fundació Centre de Recerca en Agrotecnologia
   No s'ha trobat l'entitat Fundació Centre de Recerca en Agrotecnologia
Fundació Centre de Recerca en Economia i Desenvolupament Agroalimentari UPC-IRTA


In [ ]:
import zipfile
import os
from google.colab import files

log = pd.DataFrame(log)
log.to_excel("log.xlsx")

time.sleep(30)

# Carpeta de la qual vols fer el ZIP
folder_to_zip = '/content/'

# Nom del fitxer ZIP
zip_filename = 'wipo-excels-colab.zip'

# Llista per emmagatzemar els arxius de la carpeta /content/
files_to_zip = [f for f in os.listdir(folder_to_zip) if os.path.isfile(os.path.join(folder_to_zip, f))]

# Crear el fitxer ZIP
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        file_path = os.path.join(folder_to_zip, file)
        zipf.write(file_path, arcname=file)

# Descarregar el fitxer ZIP
files.download(zip_filename)
